<a href="https://colab.research.google.com/github/AlexZ-25/BlockchainDevelopment/blob/main/Curso%20Completo%20de%20Blockchain%20de%20cero%20a%20experto/Blockchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalaciones

In [ ]:
!pip install flask==0.12.2

     |████████████████████████████████| 83 kB 916 kB/s 
  Attempting uninstall: flask
    Found existing installation: Flask 1.1.4
    Uninstalling Flask-1.1.4:
      Successfully uninstalled Flask-1.1.4


In [3]:
!pip install flask-ngrok==0.0.25

# Mi Blockchain con Python

---




In [4]:
# Importación de las librerías
import datetime
import hashlib
import json
from flask        import Flask, jsonify
from flask_ngrok  import run_with_ngrok

Creación de una clase Blockchain que contenga todos los métodos esenciales, como son los siguientes:


*   Creación de un nuevo bloque
*   Obtención del hash de un bloque
*   Protocolo de concenso Proof of Work (PoW)
*   Generación del hash de un bloque
*   Verificación de la validez de la Blockchain



In [5]:
# Creación de la Blockchain
class Blockchain:
  
  def __init__(self):
    """ Constructor de la clase Blockchain. """

    self.chain = []
    self.create_block(proof = 1, previous_hash = '0')

  def create_block(self, proof, previous_hash):
    """ Creación de un nuevo bloque.

      Arguments:
        - proof: Nounce del bloque actual. (proof != hash)
        - previous_hash: Hash del bloque previo.

      Returns:
        - block: Nuevo bloque creado.
      """
    
    block = { 'index'         : len(self.chain) + 1,
              'timestamp'     : str(datetime.datetime.now()),
              'proof'         : proof,
              'previous_hash' : previous_hash}
    self.chain.append(block)
    return block

  def get_previous_block(self):
    """ Obtención del bloque previo de la Blockchain.

        Returns:
          - Obtención del último bloque de la Blockchain. """

    return self.chain[-1]

  def proof_of_work(self, previous_proof):
    """ Protocolo de concenso Proof of Work (PoW).

      Arguments:
        - previous_proof: Nounce del bloque previo.

      Returns:
        - new_proof: Devolución del nuevo nounce obtenido con PoW. """
        
    new_proof = 1
    check_proof = False
    while check_proof is False:
      hash_operation = hashlib.sha256(str(new_proof**2 - previous_proof**2).encode()).hexdigest()
      if hash_operation[:4] == "0000":
          check_proof = True
      else:
          new_proof += 1
    return new_proof

  def hash(self, block):
    """ Cálculo del hash de un bloque.

      Arguments:
        - block: Identifica a un bloque de la Blockchain.

      Returns:
        - hash_block: Devuelve el hash del bloque. """

    encoded_block = json.dumps(block, sort_keys= True).encode()
    hash_block = hashlib.sha256(encoded_block).hexdigest()
    return hash_block

  def is_chain_valid(self, chain):
    """ Determina si la Blockchain es válida.

      Arguments:
          - chain: Cadena de bloques que contiene la información 
                    de todas las transacciones.

      Returns:
          - True/False: Devuelve un booleano en función de la validez de la 
                        Blockchain. (True = Válida, False = Inválida)"""

    previous_block = chain[0]
    block_index = 1   
    while block_index < len(chain):
        block = chain[block_index]
        if block['previous_hash'] != self.hash(previous_block):
            return False
        previous_proof = previous_block['proof']
        proof = block['proof']
        hash_operation = hashlib.sha256(str(proof**2 - previous_proof**2).encode()).hexdigest()
        if hash_operation[:4] != '0000':
            return False
        previous_block = block
        block_index += 1
    return True

Aplicación Web accesible vía en modo de REST API. Mediante llamadas HTTP al REST API podemos establecer una comunicación con el uso del módulo de Flask.

Llamadas a realizar vía REST API:


*   Minación de bloques: mine_block()
*   Obtención de la Blockchain: get_chain()
*   Comprobar estado de la Blockchain: is_valid()



In [13]:
# Crear una aplicación web
# Ejecución de la app con Notebook
app = Flask(__name__)
run_with_ngrok(app)

# Si se obtiene un error 500, actualizar flask y ejecutar la siguiente línea
app.config['JSONIFY_PRETTYPRINT_REGULAR'] = False

# Creación de la Blockchain
blockchain = Blockchain()

@app.route('/mine_block', methods =['GET'])
def mine_block():
  """ Minado de un nuevo bloque """

  previous_block = blockchain.get_previous_block()
  previous_proof = previous_block['proof']
  proof = blockchain.proof_of_work(previous_proof)
  previous_hash = blockchain.hash(previous_block)
  block = blockchain.create_block(proof, previous_hash)
  response = {'message'       : 'Enhorabuena, has minado un nuevo bloque!',
              'index'         : block['index'],
              'timestamp'     : block['timestamp'],
              'proof'         : block['proof'],
              'previous_hash' : block['previous_hash']}
  return jsonify(response), 200

@app.route('/get_chain', methods = ['GET'])
def get_chain():
  """ Obtención de la Blockchain """
  response = {'chain'   : blockchain.chain,
              'length'  : len(blockchain.chain)}
  return jsonify(response), 200

@app.route('/is_valid', methods = ['GET'])
def is_valid():
  """ Comprobación si la Blockchain es válida """

  is_valid = blockchain.is_chain_valid(blockchain.chain)
  if is_valid:
      response = {'message' : 'La cadena de bloques es válida!'}
  else:
      response = {'message' : 'La cadena de bloques NO es válida!'}
  return jsonify(response), 200

In [ ]:
# Ejecución de la app con Google Colab
app.run()

# Ejecución externa a Google colab
# app.run(host = '0.0.0.0', port = 5000)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://4117-35-229-217-54.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [23/Jan/2022 07:00:00] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [23/Jan/2022 07:00:01] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [23/Jan/2022 07:00:24] "GET /mine_block HTTP/1.1" 200 -
127.0.0.1 - - [23/Jan/2022 07:00:27] "GET /mine_block HTTP/1.1" 200 -
127.0.0.1 - - [23/Jan/2022 07:00:28] "GET /mine_block HTTP/1.1" 200 -
127.0.0.1 - - [23/Jan/2022 07:00:29] "GET /mine_block HTTP/1.1" 200 -
127.0.0.1 - - [23/Jan/2022 07:00:30] "GET /mine_block HTTP/1.1" 200 -
127.0.0.1 - - [23/Jan/2022 07:00:31] "GET /mine_block HTTP/1.1" 200 -
127.0.0.1 - - [23/Jan/2022 07:00:32] "GET /mine_block HTTP/1.1" 200 -
127.0.0.1 - - [23/Jan/2022 07:00:33] "GET /mine_block HTTP/1.1" 200 -
127.0.0.1 - - [23/Jan/2022 07:00:33] "GET /mine_block HTTP/1.1" 200 -
127.0.0.1 - - [23/Jan/2022 07:00:34] "GET /mine_block HTTP/1.1" 200 -
127.0.0.1 - - [23/Jan/2022 07:00:35] "GET /mine_block HTTP/1.1" 200 -
127.0.0.1 - - [23/Jan/2022 07:00:36] "GET /mine_block HTTP/1.1" 200 -
127.0.0.1 - - [23/Jan/2022 07:00:37] "GE